In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics
import geopandas as gpd
import intake
import pandas as pd

from typing import Literal

from calitp_data_analysis import geography_utils
from calitp_data_analysis import calitp_color_palette as cp
from bus_service_utils import better_bus_utils

catalog = intake.open_catalog("../bus_service_increase/*.yml")

In [ ]:
# parameters cell
district = "07 - Los Angeles"

In [ ]:
gdf = catalog.highway_uncompetitive_transit.read()

gdf = better_bus_utils.add_district_description(gdf)
gdf = gdf[gdf.caltrans_district == district]

In [ ]:
%%capture_parameters
district_num = str(gdf.District.iloc[0])
district = gdf.caltrans_district.iloc[0]
district, district_num

# Highways with Zero or Uncompetitive Transit

In [ ]:
# Map args
TILES = "CartoDB Positron"

def data_for_viz(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    gdf2 = (gdf.drop(columns = ["hwy_segment_id", 
                                "segment_sequence", "_merge"])
            .rename(columns = {"competitive_category": "Category"})
           )
    
    return gdf2

def make_transit_desert_or_uncompetitive_map(gdf: gpd.GeoDataFrame): 
    include = ['no transit', 'uncompetitive transit only']
    gdf2 = data_for_viz(gdf[gdf.competitive_category.isin(include)])
    
    colors = (cp.CALITP_CATEGORY_BRIGHT_COLORS + 
          cp.CALITP_CATEGORY_BOLD_COLORS + 
          cp.CALITP_DIVERGING_COLORS
         )
    
    if len(gdf2) > 0:
        m = gdf2.explore("Route", 
                         categorical = True, 
                         cmap = colors, 
                         tiles = TILES
                        )
    else:
        display(Markdown(
            f"All highways in District {district_num} have some competitive transit available."
        ))
        m = None
    
    return m


def make_competitive_category_transit_map(gdf: gpd.GeoDataFrame):
    
    gdf2 = data_for_viz(gdf)
    
    # Switch color ordering 
    colors = [
        cp.CALITP_CATEGORY_BRIGHT_COLORS[3], # green
        cp.CALITP_CATEGORY_BRIGHT_COLORS[2], # yellow
        cp.CALITP_CATEGORY_BRIGHT_COLORS[1], # orange
    ]
            
    m = gdf2.explore("Category", 
                     categorical = True, tiles = TILES, 
                     cmap = colors
                    )
    
    return m

## Highways with Competitive / Uncompetitive / Zero Transit

In [ ]:
m = make_competitive_category_transit_map(gdf)
m

## Transit Deserts

In [ ]:
m = make_transit_desert_or_uncompetitive_map(gdf)
m